In [1]:
__nbid__ = '0054'
__author__ = 'Mike Fitzpatrick <mike.fitzpatrick@noirlab.edu>, Robert Nikutta <robert.nikutta@noirlab.edu>'
__version__ = '20240607'
__datasets__ = []
__keywords__ = []

## Cómo usar el Servicio Data Lab * Store Client *

Este notebook documenta cómo usar el sistema de almacenamiento virtual de Data Lab a través del servicio al cliente de la tienda. Esto se puede hacer desde un script de Python (por ejemplo, dentro de este notebook) o desde la línea de comando usando el comando <i> DataLab </i>.

## Descargo de responsabilidad y atribución

Renuncias
-----------
Tenga en cuenta que el uso del laboratorio de datos Astro constituye su acuerdo con nuestros mínimos [renunciadores](https://datalab.noirlab.edu/disclaimers.php).

Expresiones de gratitud
---------------
Si usa ** Astro Data Lab ** En su investigación publicada, incluya el texto en la sección de Agradecimientos de su artículo:

_Esta investigación utiliza servicios o datos proporcionados por Astro Data Lab, que forma parte del programa de Centro de Ciencias y Datos de la Comunidad (CSDC) de NSF Noirlab. Noirlab es operado por la Asociación de Universidades para la Investigación en Astronomía (Aura), Inc. bajo un acuerdo cooperativo con la Fundación Nacional de Ciencias de EE. UU.

Si usa ** Sparcl conjuntamente con la plataforma Astro Data Lab Platform ** (a través de Jupyterlab, línea de comandos o interfaz web) En su investigación publicada, incluya este texto a continuación en la sección de Agradecimientos de su artículo:

_ Esta investigación utiliza servicios o datos proporcionados por el análisis de análisis de espectros y el laboratorio de catálogo recuperable (SPARCL) y el Astro Data Lab, que forman parte del programa de Centro de Ciencia y Datos de la Comunidad (CSDC) de NSF Noirlab. Noirlab es operado por la Asociación de Universidades para la Investigación en Astronomía (Aura), Inc. bajo un acuerdo cooperativo con la Fundación Nacional de Ciencias de EE. UU.

En cualquier caso ** Cite los siguientes documentos **:

* Documento de concepto de laboratorio de datos: Fitzpatrick et al., "The Noao Data Laboratory: una visión general conceptual", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Descripción general del laboratorio de datos Astro: Nikutta et al., "Laboratorio de datos: una plataforma de ciencia comunitaria", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

Si se refiere al laboratorio de datos JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Análisis astrofísico habilitado para Jupyter utilizando plataformas informáticas de datos con datos", Cise, 23, 15, 2021, https://doi.org/10.1109/mcse.2021.3057097

Si se publica en un diario de AAS, también agregue la palabra clave: `\ facilidad {Astro Data Lab}`

Y si está utilizando SPARCL, también agregue `\ Software {Sparcl}` y cite:

* Juneau et al., "Sparcl: análisis de espectros y laboratorio de catálogo recuperable", Actas de conferencia para Atass xxxiii, 2024
https://doi.org/10.48550/arxiv.2401.05576

La Biblioteca Noirlab mantiene [listas de reconocimientos adecuados](https://noirlab.edu/science/about/scientific-acknowledgments) para usar al publicar documentos utilizando las instalaciones, datos o servicios del laboratorio.

### La interfaz de servicio de Storage Manager

El servicio al cliente de la tienda simplifica el acceso al sistema de almacenamiento virtual del laboratorio de datos. Esta sección describe la interfaz de servicio al cliente de la tienda en caso de que queramos escribir nuestro propio código contra eso en lugar de usar una de las herramientas proporcionadas. El servicio al cliente de la tienda acepta una llamada HTTP Get al punto final apropiado para la operación en particular:

| Punto final | Descripción | Parámetros req'd |
| ---------- | ------------- | ------------ |
| /Get | Recuperar un archivo | nombre |
| /PUT | Subir un archivo | nombre |
| /Cargar | Cargue un archivo en Vospacio | Nombre, punto final |
| /CP | Copiar un archivo/directorio | de, a |
| /ln | Enlace un archivo/directorio | de, a |
| /Lock | Bloquear un nodo de las actualizaciones de escritura | nombre |
| /ls | Obtenga un archivo/listado de directorio | nombre |
| /Acceso | Determinar la accesibilidad de archivos | nombre |
| /STAT | Información de estado del archivo | Nombre, detallado |
| /Mkdir | Crear un directorio | nombre |
| /MV | Mover/cambiar el nombre de un archivo/directorio | de, a |
| /RM | Eliminar un archivo | nombre |
| /rmdir | Eliminar un directorio | nombre |
| /etiqueta | Anotar un archivo/directorio | Nombre, etiqueta |

Por ejemplo, una llamada a <i> http://datalab.noirlab.edu/storage/get?name=vos://mag.csv </i> recuperará el archivo '_mag.csv_' del directorio raíz del almacenamiento virtual del usuario.  Del mismo modo, una llamada de Python usando la interfaz _storeclient_ como "_storeclient.get ('vos: //mag.csv') _" obtendría el mismo archivo.

#### Identificadores de almacenamiento virtual

Los archivos en el almacenamiento virtual generalmente se identifican a través del prefijo "_vos: // _". Este identificador de taquigrafía se resuelve con el directorio de inicio del usuario del espacio de almacenamiento en el servicio.  Como conveniencia, el prefijo se puede omitir opcionalmente cuando el parámetro se refiere a un nodo en el almacenamiento virtual. La navegación por encima del directorio de inicio de un usuario no es compatible, sin embargo, los subdirectorios dentro del espacio pueden crearse y usarse según sea necesario.

#### Autenticación
El servicio de Storage Manager requiere un token de seguridad de datos. Esto debe aprobarse como el valor de la palabra clave del encabezado "X-DL-AuthToken" en cualquier llamada HTTP al servicio. Si el token no se suministra, se supone el acceso anónimo, pero proporciona acceso solo a los espacios de almacenamiento público.

### del código Python

El servicio al cliente de la tienda se puede llamar desde el código Python utilizando el módulo <i> DataLab </i>. Esto proporciona métodos para acceder a las diversas funciones en el subpackage <i> StoreClient </i>. 

#### Inicialización
Esta es la configuración que se requiere para usar el cliente de la tienda. Lo primero que debe hacer es importar los módulos Python relevantes y también recuperar nuestro token de seguridad de datos.

In [2]:
# Importaciones estándar de notebook
from getpass import getpass
from dl import authClient, storeClient

Comente y ejecute la celda a continuación si necesita iniciar sesión en Data Lab:

In [3]:
# # Obtenga el token de autenticación para el usuario
# token = authClient.login (input ("Ingrese el nombre de usuario: (+Enter)"), GetPass ("Ingrese la contraseña: (+Enter)"))
# Si no authclient.IsValidToken (token):
# elevar la excepción ('El token no es válido. Verifique su nombre de usuario/contraseña y ejecute esta celda nuevamente').

#### enumerando un archivo/directorio

Podemos ver todos los archivos que están en un directorio específico o obtener una lista completa para un archivo específico.  En este caso, enumeraremos el directorio de almacenamiento virtual predeterminado que utilizará como base para los cambios que realizaremos a continuación.

In [4]:
listing = storeClient.ls (name = 'vos://')
print (listing)

cutout.fits,public,results,tmp


El directorio * público * que se muestra aquí es visible para todos los usuarios del laboratorio de datos y proporciona un medio para compartir datos sin tener que configurar el acceso especial.  Del mismo modo, el directorio * TMP * se lee protegido y proporciona un directorio temporal conveniente que se utilizará en un flujo de trabajo.

#### Existencia e información del archivo

Además de simplemente enumerar archivos, es posible probar si un archivo con nombre ya existe o determinar más información al respecto.

In [5]:
# Una prueba de existencia de archivo simple:
if storeClient.access ('vos://public'):
    print ('Usuario"public"Existe el directorio')
if storeClient.access ('vos://public', mode='w'):
    print ('Usuario"public"El directorio es grupal/World Writable')
else:
    print ('Usuario"public"El directorio no es grupal/mundial WRITITY')
    
if storeClient.access ('vos://tmp'):
    print ('Usuario"tmp"Existe el directorio')        
if storeClient.access ('vos://tmp', mode='w'):
    print ('Usuario"tmp"El directorio es grupal/World Writable')
else:
    print ('Usuario"tmp"El directorio no es grupal/mundial WRITITY')

User "public" directory exists
User "public" directory is not group/world writable
User "tmp" directory exists
User "tmp" directory is not group/world writable


#### Subir un archivo

Ahora queremos cargar un nuevo archivo de datos de nuestro disco local al almacenamiento virtual:

In [6]:
storeClient.put (to = 'vos://newmags.csv', fr = './newmags.csv')
print(storeClient.ls (name='Vos: //'))

(1 / 1) ./newmags.csv -> vos://newmags.csv
cutout.fits,newmags.csv,public,results,tmp


#### descargar un archivo

Supongamos que queremos descargar un archivo de nuestro espacio de almacenamiento virtual, en este caso, un resultado de consulta que le guardamos en el notebook "Cómo usar el servicio de consulta del laboratorio de datos".

In [7]:
storeClient.get (fr = 'vos://newmags.csv', to = './mymags.csv')

(1/1) [====================] [   142B] newmags.csv


['OK']

También es posible obtener el contenido de un archivo remoto directamente en su notebook especificando la ubicación como una cadena vacía:

In [8]:
data = storeClient.get (fr = 'vos://newmags.csv', to = '')
print (data)

id,g,r,i
001,22.3,12.4,21.5
002,22.3,12.4,21.5
003,22.3,12.4,21.5
004,22.3,12.4,21.5
005,22.3,12.4,21.5
006,22.3,12.4,21.5
007,22.3,12.4,21.5



#### Carga de un archivo desde una URL remota

Es posible cargar un archivo directamente al almacenamiento virtual desde una URL remota), por ejemplo. un "accessurl" para un corte de imagen, un archivo de datos remoto, etc.) utilizando el método "storeClient.load ()":

In [9]:
url = "http://datalab.noirlab.edu/svc/cutout?col=&siaRef=c4d_161005_022804_ooi_g_v1.fits.fz&extn=31&POS=335.0,0.0&SIZE=0.1"
storeClient.load('vos://cutout.fits',url)

'OK'

#### Creación de un directorio

Podemos crear un directorio en el almacenamiento remoto que se utilizará para guardar datos más adelante:

In [10]:
storeClient.mkdir ('vos://results')

'A Node already exists with the requested URI.'

#### Copiar un archivo/directorio

Queremos poner una copia del archivo en un directorio de trabajo remoto:

In [11]:
storeClient.mkdir ('vos://temp')
print ("Antes:" + storeClient.ls (name='Vos: // temp/'))
storeClient.cp (fr = 'vos://newmags.csv', to = 'vos://temp/newmags.csv',verbose=True)
print ("Después:" + storeClient.ls (name='Vos: // temp/'))

Before: 
After: newmags.csv


In [12]:
print(storeClient.ls('Vos: //',format='largo'))

-rw-rw-r-x  demo01  2963520  22 Nov 2021 14:22  cutout.fits
-rw-rw-r-x  demo01     142  30 Nov 2021 14:58  newmags.csv
drwxrwxr-x  demo01       0  14 Jul 2020 10:01  public/
drwxrwxr-x  demo01       0  22 Nov 2021 14:22  results/
drwxrwxr-x  demo01       0  30 Nov 2021 14:58  temp/
drwxrwx---  demo01       0  14 Jul 2020 10:01  tmp/



Observe que en el * ls () * Llama agregamos el nombre del directorio con un final '/' para enumerar el contenido del directorio en lugar del directorio en sí.

#### vinculado a un archivo/directorio

** ADVERTENCIA **: El enlace actualmente ** no ** trabajando en el administrador de almacenamiento de laboratorio de datos. Este notebook se actualizará cuando se haya resuelto el problema.

A veces queremos crear un enlace a un archivo o directorio.  En este caso, el enlace nombrado por el parámetro * 'fr' * se crea y señala el archivo/contenedor nombrado por el parámetro * 'objetivo' *.

In [13]:
storeClient.ln ('vos://mags.csv', 'vos://temp/newmags.csv')
print ("Dir de raíz:" + storeClient.ls (name='Vos: //'))
print ("Temper Dir:" + storeClient.ls (name='Vos: // temp/'))

Root dir: cutout.fits,newmags.csv,public,results,temp,tmp
Temp dir: newmags.csv


#### Mover/renombrar un archivo/directorio

Podemos mover un archivo o directorio:

In [14]:
storeClient.mv(fr = 'vos://temp/newmags.csv', to = 'vos://results')
print ("Resultados Dir:" + storeClient.ls (name='Vos: // resultados/'))

Results dir: newmags.csv


#### Eliminar un archivo

Podemos eliminar un archivo:

In [15]:
print ("Antes:" + storeClient.ls (name='Vos: //'))
storeClient.rm (name = 'vos://mags.csv')
print ("Después:" + storeClient.ls (name='vos://'))

Before: cutout.fits,newmags.csv,public,results,temp,tmp
After: cutout.fits,newmags.csv,public,results,temp,tmp


#### Eliminar un directorio

También podemos eliminar un directorio, hacerlo también elimina el contenido de ese directorio:

In [16]:
storeClient.rmdir(name = 'vos://temp')

'OK'

#### etiquetando un archivo/directorio

** ADVERTENCIA **: El etiquetado actualmente ** no ** trabajando en el administrador de almacenamiento de laboratorio de datos. Este notebook se actualizará cuando se haya resuelto el problema.

Podemos etiquetar cualquier archivo o directorio con metadatos arbitrarios:

In [17]:
storeClient.tag('vos://results', 'The results from my analysis')

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request.  Either the server is overloaded or there is an error in the application.</p>\n'

#### limpieza el directorio de demostración de los archivos restantes

In [18]:
storeClient.rm (name = 'vos://newmags.csv')
storeClient.rm (name = 'vos://results')
storeClient.ls (name = 'vos://')

'cutout.fits,public,results,tmp'

### usando el comando Datalab

El comando <i> DataLab </i> proporciona una forma de comando alternativa para funcionar con el administrador de consultas a través de los subcomands <i> consulta </i>, que es especialmente útil si desea interactuar con el administrador de consultas desde su computadora local. Tenga la utilidad de línea de comandos `Datalab` instalada primero (para instalar instrucciones, consulte https://github.com/astro-datalab/datalab).

Se comentan las celdas a continuación. Copie y pegue cualquiera de ellos (sin el signo de comentarios) y ejecute localmente.

#### Inicie sesión una vez

In [19]:
# ! Inicio de sesión de Datalab

e ingrese las credenciales según lo solicitado.

#### Descargando un archivo

Digamos que queremos descargar un archivo de nuestro espacio de almacenamiento virtual:

In [20]:
# ! Datalab Get fr = "vos: //mags.csv" to = "./ mags.csv"

#### Subir un archivo

Ahora queremos cargar un nuevo archivo de datos desde nuestro disco local:

In [21]:
# ! Datalab Put fr = "./ newmags.csv" a = "vos: //newsmags.csv"

#### Copiar un archivo/directorio

Queremos poner una copia del archivo en un directorio de trabajo remoto:

In [22]:
# ! Datalab cp fr = "vos: //newsmags.csv" a = "vos: //temp/newsmags.csv"

#### vinculado a un archivo/directorio

A veces queremos crear un enlace a un archivo o directorio:

In [23]:
# ! Datalab ln fr = "vos: //temp/mags.csv" to = "vos: //mags.csv"

#### enumerando un archivo/directorio

Podemos ver todos los archivos que están en un directorio específico o obtener una lista completa para un archivo específico:

In [24]:
# ! Datalab ls name = "Vos: // temp"

#### Creación de un directorio

Podemos crear un directorio:

In [25]:
# ! Datalab mkdir name = "Vos: // Resultados"

#### Mover/renombrar un archivo/directorio

Podemos mover un archivo o directorio:

In [26]:
# ! Datalab mv fr = "Vos: //temp/newsmags.csv" a = "Vos: // Resultados"

#### Eliminar un archivo

Podemos eliminar un archivo:

In [27]:
# ! Datalab rm name = "Vos: //temp/mags.csv"

#### Eliminar un directorio

También podemos eliminar un directorio:

In [28]:
# ! Datalab rmdir name = "Vos: // temp"

#### etiquetando un archivo/directorio

Podemos etiquetar cualquier archivo o directorio con metadatos arbitrarios:

In [29]:
# ! DataLab Tag Name = "Vos: // Results" tag = "Los resultados de mi análisis"